In [1]:
from utils.gaussian_viewing_utils import * 
import matplotlib.pyplot as plt
import os
from moviepy.editor import ImageSequenceClip
from gaussian_renderer import render
from scene.gaussian_model import GaussianModel
from scene.cameras import Camera as GSCamera
#import kaolin

#def log_tensor(t, name, **kwargs):
#    print(kaolin.utils.testing.tensor_info(t, name=name, **kwargs))

In [3]:
model_path = '/raid/scratch_not_backed_up/sbonilla/gaussian_ready_c3v4/mip-output-noeval-no-factor'

# Number of views
num_views = 140

# Camera ID
novel_view_camera_id = 3   

# Mask visualization output
output_path = '/home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/scaling-masks.mp4'

# Novel view visualization output
nv_output_path = f'/raid/scratch_not_backed_up/sbonilla/gaussian_ready_c3v4/mip-output-noeval-no-factor/visualization/novel-views-around-camera-id{novel_view_camera_id}-y.mp4'

In [4]:
class PipelineParamsNoparse:
    """ Same as PipelineParams but without argument parser. """
    def __init__(self):
        self.convert_SHs_python = False
        self.compute_cov3D_python = False                      
        self.debug = False

        
def load_checkpoint(model_path, sh_degree=3, iteration=-1):
    # Find checkpoint
    checkpt_dir = os.path.join(model_path, "point_cloud")
    if iteration == -1:
        iteration = searchForMaxIteration(checkpt_dir)
    checkpt_path = os.path.join(checkpt_dir, f"iteration_{iteration}", "point_cloud.ply")
    
    # Load guassians
    gaussians = GaussianModel(sh_degree)
    gaussians.load_ply(checkpt_path)                                                 
    return gaussians


def try_load_camera(model_path, idx=0):
    """ Load one of the default cameras for the scene. """
    cam_path = os.path.join(model_path, 'cameras.json')
    if not os.path.exists(cam_path):
        print(f'Could not find saved cameras for the scene at {cam_path}; using default for ficus.')
        return GSCamera(colmap_id=0,
                        R=np.array([[-9.9037e-01,  2.3305e-02, -1.3640e-01], [ 1.3838e-01,  1.6679e-01, -9.7623e-01], [-1.6444e-09, -9.8571e-01, -1.6841e-01]]), 
                        T=np.array([6.8159e-09, 2.0721e-10, 4.03112e+00]), 
                        FoVx=0.69111120, FoVy=0.69111120, 
                        image=torch.zeros((3, 800, 800)),  # fake 
                        gt_alpha_mask=None, image_name='fake', uid=0)
        
    with open(cam_path) as f:
        data = json.load(f)
        raw_camera = data[idx]
        
    tmp = np.zeros((4, 4))
    tmp[:3, :3] = raw_camera['rotation']
    tmp[:3, 3] = raw_camera['position']
    tmp[3, 3] = 1
    name = raw_camera['img_name']
    C2W = np.linalg.inv(tmp)
    R = C2W[:3, :3].transpose()
    T = C2W[:3, 3]
    width = raw_camera['width']
    height = raw_camera['height']
    fovx = focal2fov(raw_camera['fx'], width)
    fovy = focal2fov(raw_camera['fy'], height)
    return GSCamera(colmap_id=0,
                    R=R, T=T, FoVx=fovx, FoVy=fovy, 
                    image=torch.zeros((3, height, width)),  # fake 
                    gt_alpha_mask=None, image_name=name, uid=0)

def load_camera_map(camera_file, name_convention='id.png'):
    with open(camera_file, 'r') as f:
        data = json.load(f)
    if name_convention == 'id.png':
        return {int(cam['img_name']): cam['id'] for cam in data}
    elif name_convention == 'id_color.jpg':
        return {int(cam['img_name'].split('_')[0]): cam['id_color'] for cam in data}
    else:
        raise ValueError('name_convention not recognized')

gaussians = load_checkpoint(model_path)
pipeline = PipelineParamsNoparse()
background = torch.tensor([0, 0, 0], dtype=torch.float32, device="cuda")
center_camera = try_load_camera(model_path, idx=novel_view_camera_id)

In [5]:
!nvidia-smi

Thu Nov 30 15:48:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   32C    P0    74W / 400W |  24766MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:0F:00.0 Off |                    0 |
| N/A   

In [6]:
!export CUDA_VISIBLE_DEVICES=6

In [8]:
#img_array = render_mask_gaussians(model_path, percent_of_gaussians=0.6, camera_index=novel_view_camera_id, kernel_size=0.1)
#img = Image.fromarray(img_array, 'RGB')
#plt.imshow(img)

AttributeError: 'GaussianModel' object has no attribute 'filter_3D'

In [7]:
render_rotate_about_x_path_video(model_path, novel_view_camera_id, 5, 20, nv_output_path, num_steps=60, num_repeats=1, kernel_size=0.1)

Moviepy - Building video /raid/scratch_not_backed_up/sbonilla/gaussian_ready_c3v4/mip-output-noeval-no-factor/visualization/novel-views-around-camera-id3-z.mp4.
Moviepy - Writing video /raid/scratch_not_backed_up/sbonilla/gaussian_ready_c3v4/mip-output-noeval-no-factor/visualization/novel-views-around-camera-id3-z.mp4



Moviepy - Done !
Moviepy - video ready /raid/scratch_not_backed_up/sbonilla/gaussian_ready_c3v4/mip-output-noeval-no-factor/visualization/novel-views-around-camera-id3-z.mp4
Video saved to /raid/scratch_not_backed_up/sbonilla/gaussian_ready_c3v4/mip-output-noeval-no-factor/visualization/novel-views-around-camera-id3-z.mp4


In [18]:
images = []

cam_path = os.path.join(model_path, 'cameras.json')
camera_idx_map = load_camera_map(cam_path)

for i in range(num_views):
    camera_idx = camera_idx_map.get(i)
    if camera_idx is not None:
        img_array = render_mask_gaussians(model_path, percent_of_gaussians=i/num_views, camera_index=camera_idx)
        images.append(img_array)

In [20]:
# create video from all frames in full_path using moviepy
clip = ImageSequenceClip(images, fps=15)
clip.write_videofile(output_path, codec='libx264', audio=False)
print(f'Video saved to {output_path}')

Moviepy - Building video /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/scaling-masks.mp4.
Moviepy - Writing video /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/scaling-masks.mp4



Moviepy - Done !
Moviepy - video ready /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/scaling-masks.mp4
Video saved to /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/scaling-masks.mp4


In [3]:
render_circular_conical_path_video(model_path=model_path, 
                                   num_views=num_views, 
                                   center_camera_idx=novel_view_camera_id, 
                                   output_path=nv_output_path, 
                                   tilt_angle=10, 
                                   fps=15)

ValueError: cannot reshape array of size 3 into shape (3,3)

In [26]:
spiral = generate_spiral_cam_to_world(radii=[0.1, 0.1, 0.1], focus_depth=center_camera.FoVx, n_poses=120)

In [27]:
cameras = []
cameras.append(center_camera)
for idx, row in enumerate(spiral):
    new_R = np.matmul(row[:, :3], center_camera.R)
    new_T = row[:, 3] + center_camera.T
    cameras.append(GSCamera(colmap_id=idx,
                            R=new_R, T=new_T, FoVx=center_camera.FoVx, FoVy=center_camera.FoVy, 
                            image=torch.zeros((3, center_camera.image_height, center_camera.image_width)),  
                            gt_alpha_mask=None, image_name='fake', uid=0))

find mean of all points in points3D

In [28]:
images = []
for camera in cameras:
    render_res = render(camera, gaussians, pipeline, background)
    rendering = render_res["render"]
    image = (rendering.permute(1, 2, 0) * 255).to(torch.uint8).detach().cpu().numpy()
    images.append(image)

In [29]:
clip = ImageSequenceClip(list(images), fps=20)
clip.write_videofile(nv_output_path, codec='libx264', audio=False)
print(f'Video saved to {nv_output_path}')

Moviepy - Building video /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/novel-views-around-camera-id0.mp4.
Moviepy - Writing video /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/novel-views-around-camera-id0.mp4



Moviepy - Done !
Moviepy - video ready /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/novel-views-around-camera-id0.mp4
Video saved to /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/novel-views-around-camera-id0.mp4


In [7]:
center_camera_idx = 50
fps = 20
output_path = f'/home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/spiral-path.mp4'
render_spiral_path_video(model_path, center_camera_idx, fps, output_path, n_poses=120, n_rotations=2, radius=4, iteration=-1, sh_degree=3)

Moviepy - Building video /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/spiral-path.mp4.
Moviepy - Writing video /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/spiral-path.mp4



Moviepy - Done !
Moviepy - video ready /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/spiral-path.mp4
Video saved to /home/sbonilla/gaussian-splatting/output/f8df57c9-c/visualization/spiral-path.mp4
